In [1]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib.pyplot import cm
import json

In [2]:
import sys
print(sys.path) 
sys.path.append("/cellar/users/mpagadal/Programs/anaconda3/lib/python3.7/site-packages")
sys.path.insert(1, '/cellar/users/mpagadal/Data/gcta/scripts')

['/data/cellardata2/users/mpagadal/projects/germline-immune/cell-type-specificity/notebooks', '/nrnb/opt/anaconda3.7/lib/python37.zip', '/nrnb/opt/anaconda3.7/lib/python3.7', '/nrnb/opt/anaconda3.7/lib/python3.7/lib-dynload', '', '/cellar/users/mpagadal/.local/lib/python3.7/site-packages', '/nrnb/opt/anaconda3.7/lib/python3.7/site-packages', '/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/IPython/extensions', '/cellar/users/mpagadal/.ipython']


In [3]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## make random snps dataframe

### get genes that are found in tcga

In [4]:
genes=pd.read_csv("/cellar/users/mpagadal/Data/projects/germline-immune/discovery/phenotypes/processed_phenos/tpm/all.rna.tsv",delimiter="\t",usecols=[0])
genes2=pd.read_csv("/cellar/users/mpagadal/Data/projects/germline-immune/discovery/phenotypes/processed_phenos/pancanatlas/all.rna.tsv",delimiter="\t",usecols=[0])
genes3=pd.read_csv("/cellar/users/mpagadal/Data/projects/germline-immune/discovery/phenotypes/processed_phenos/firebrowse/all.rna.tsv",delimiter="\t",usecols=[0])

In [5]:
genes=[x for x in genes["Gene"].tolist() if x in genes2["Gene"].tolist()]
genes=[x for x in genes if x in genes3["Gene"].tolist()]

In [6]:
with open('/cellar/users/mpagadal/Data/resources/tcga/ensembl_map.json', 'r') as f:
    ensembl_dict = json.load(f)

In [7]:
ensembl_dict={v:k.split(".")[0] for k,v in ensembl_dict.items()}

In [8]:
ids=[]

for x in genes:
    try:
        ids.append(ensembl_dict[x])
    except:
        pass

In [9]:
len(genes)

17579

In [10]:
mp=pd.read_csv("/cellar/users/mpagadal/Data/gtex/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.lookup_table.txt",delimiter="\t")
mp_b38_b37=dict(zip(mp.variant_id,mp.variant_id_b37))

In [11]:
compiled=pd.DataFrame()

for x in [x for x in range(1,23)]+["X"]:
    print(x)
    eqtl=pd.read_parquet("/cellar/users/mpagadal/Data/gtex/GTEx_Analysis_v8_QTLs/GTEx_Analysis_v8_EUR_eQTL_all_associations/Breast_Mammary_Tissue.v8.EUR.allpairs.chr"+str(x)+".parquet",engine="pyarrow")
    eqtl["gene_id"]=eqtl["phenotype_id"].str.split(".").str[0]
    eqtl_filt=eqtl[eqtl["gene_id"].isin(ids)]
    compiled=compiled.append(eqtl_filt)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
X


In [12]:
compiled.head()

,phenotype_id,variant_id,tss_distance,maf,ma_samples,ma_count,pval_nominal,slope,slope_se,gene_id
0,ENSG00000187634.11,chr1_13550_G_A_b38,-910378,0.012158,8,8,0.443292,-0.169791,0.221152,ENSG00000187634
1,ENSG00000187634.11,chr1_14671_G_C_b38,-909257,0.010638,7,7,0.955603,0.012550,0.225210,ENSG00000187634
2,ENSG00000187634.11,chr1_14677_G_A_b38,-909251,0.048632,32,32,0.795417,0.027932,0.107624,ENSG00000187634
3,ENSG00000187634.11,chr1_16841_G_T_b38,-907087,0.048632,32,32,0.313947,-0.107930,0.106985,ENSG00000187634
4,ENSG00000187634.11,chr1_17005_A_G_b38,-906923,0.025836,17,17,0.281260,0.163689,0.151617,ENSG00000187634


In [13]:
compiled["variant_b37"] = compiled["variant_id"].map(mp_b38_b37)
compiled["variant"] = compiled["variant_b37"].str.split("_").str[0]+":"+compiled["variant_b37"].str.split("_").str[1]+":"+compiled["variant_b37"].str.split("_").str[2]+":"+compiled["variant_b37"].str.split("_").str[3]

### ensure snps are found in tcga

In [14]:
bim=pd.read_csv("/nrnb/users/mpagadal/tcga-genotypes/imputation/michigan-imputation/HRC/european.final.noimmunecancers.clean.bim",delimiter="\t",header=None)

In [15]:
snps=bim[1].tolist()+[x.rsplit(":",2)[0]+":"+x.split(":")[3]+":"+x.split(":")[2] for x in bim[1].tolist()]

In [16]:
compiled_filt=compiled[compiled["variant"].isin(snps)]

In [17]:
compiled_filt.head()

,phenotype_id,variant_id,tss_distance,maf,ma_samples,ma_count,pval_nominal,slope,slope_se,gene_id,variant_b37,variant
146,ENSG00000187634.11,chr1_758351_A_G_b38,-165577,0.107903,65,71,0.238529,0.082620,0.069943,ENSG00000187634,1_693731_A_G_b37,1:693731:A:G
154,ENSG00000187634.11,chr1_770988_A_G_b38,-152940,0.455927,240,300,0.127656,-0.072567,0.047490,ENSG00000187634,1_706368_A_G_b37,1:706368:A:G
170,ENSG00000187634.11,chr1_779216_T_C_b38,-144712,0.066869,44,44,0.439378,-0.073659,0.095121,ENSG00000187634,1_714596_T_C_b37,1:714596:T:C
172,ENSG00000187634.11,chr1_779885_C_T_b38,-144043,0.068389,45,45,0.459901,-0.069646,0.094109,ENSG00000187634,1_715265_C_T_b37,1:715265:C:T
173,ENSG00000187634.11,chr1_779987_A_G_b38,-143941,0.068389,45,45,0.459901,-0.069646,0.094109,ENSG00000187634,1_715367_A_G_b37,1:715367:A:G


In [18]:
compiled_filt.shape

(77006529, 12)

### randomization

In [19]:
gtex_rand=compiled_filt.sample(n=1000)

In [20]:
gtex_rand.to_csv("gtex_rand.csv",index=None,sep="\t")

### make snps dataframe

In [21]:
gtex_rand=pd.read_csv("gtex_rand.csv",delimiter="\t")

In [22]:
gtex_rand_snps=gtex_rand["variant"].tolist()+[x.rsplit(":",2)[0]+":"+x.split(":")[3]+":"+x.split(":")[2] for x in gtex_rand["variant"].tolist()]

In [23]:
len(gtex_rand)

1000

In [24]:
rand=pd.DataFrame({"snps":gtex_rand_snps})

In [25]:
rand.to_csv("extract_gtex_rand_snps.txt",header=None,index=None)

In [26]:
with open('/cellar/users/mpagadal/Data/resources/tcga/ensembl_map.json', 'r') as f:
    ensembl_dict = json.load(f)

In [27]:
ensembl_dict={k.split(".")[0]:v for k,v in ensembl_dict.items()}

In [28]:
gtex_rand["gene"]=gtex_rand["gene_id"].map(ensembl_dict)

In [29]:
len(gtex_rand["gene"].unique())

974

In [30]:
rand_genes=pd.DataFrame({"genes":gtex_rand["gene"].unique().tolist()})

In [31]:
rand_genes.to_csv("gtex_rand_genes.txt",index=None,sep="\t")

In [32]:
len(rand_genes["genes"].tolist())

974

## Get random snps

In [21]:
#directory with snps extracted from gtex parquet files
directory='./rand_extracted_snps/'
gtex_files=[x for x in os.listdir(directory) if x.endswith("gtex.cis.eur.snps.csv")]

In [22]:
#compiled snps from all gtex tissues
total_gtex=pd.DataFrame()

for x in gtex_files:
    cis=pd.read_csv(directory+x)
    cis["file"]=x.split(".gtex.cis.eur.snps.csv")[0]
    total_gtex=total_gtex.append(cis)

In [23]:
#grab snps for which ref/alt order is reversed
total_gtex["variant2"]=total_gtex["variant_b37"].str.split("_").str[0]+":"+total_gtex["variant_b37"].str.split("_").str[1]+":"+total_gtex["variant_b37"].str.split("_").str[3]+":"+total_gtex["variant_b37"].str.split("_").str[2]
#get gene ids for mapping
total_gtex["gene_id"]=total_gtex["phenotype_id"].str.split(".").str[0]
total_gtex.head()

,Unnamed: 0,phenotype_id,variant_id,tss_distance,maf,ma_samples,ma_count,pval_nominal,slope,slope_se,variant_b37,variant,file,variant2,gene_id
0,2323,ENSG00000237094.11,chr1_1352965_A_G_b38,851348,0.028846,15,15,0.895738,-0.036855,0.280882,1_1288345_A_G_b37,1:1288345:A:G,Stomach,1:1288345:G:A,ENSG00000237094
1,2323,ENSG00000225972.1,chr1_1352965_A_G_b38,723903,0.028846,15,15,0.848214,0.032457,0.169367,1_1288345_A_G_b37,1:1288345:A:G,Stomach,1:1288345:G:A,ENSG00000225972
2,2323,ENSG00000225630.1,chr1_1352965_A_G_b38,723325,0.028846,15,15,0.921226,-0.021156,0.213673,1_1288345_A_G_b37,1:1288345:A:G,Stomach,1:1288345:G:A,ENSG00000225630
3,2323,ENSG00000237973.1,chr1_1352965_A_G_b38,721891,0.028846,15,15,0.729151,0.060864,0.175537,1_1288345_A_G_b37,1:1288345:A:G,Stomach,1:1288345:G:A,ENSG00000237973
4,2323,ENSG00000229344.1,chr1_1352965_A_G_b38,720208,0.028846,15,15,0.054454,0.324285,0.167650,1_1288345_A_G_b37,1:1288345:A:G,Stomach,1:1288345:G:A,ENSG00000229344


In [24]:
#extract reference and alternate alleles
#according to gtex guide reference is first in variant id followed by alternate allele
total_gtex["ref"]=total_gtex["variant_id"].str.split("_").str[2]
total_gtex["alt"]=total_gtex["variant_id"].str.split("_").str[3]

In [25]:
total_gtex_melt=pd.melt(total_gtex, id_vars=(["gene_id","slope","slope_se","ref","alt","file","pval_nominal","maf","tss_distance","variant_id"]),value_vars=(["variant","variant2"]))

In [28]:
with open('/cellar/users/mpagadal/Data/resources/tcga/ensembl_map.json', 'r') as f:
    ensembl_dict = json.load(f)

In [29]:
ensembl_dict={k.split(".")[0]:v for k,v in ensembl_dict.items()}

In [30]:
total_gtex["gene_name"]=total_gtex["gene_id"].map(ensembl_dict)

In [34]:
len(total_gtex["gene_name"].unique())

19859

In [35]:
total_gtex.head()

,Unnamed: 0,phenotype_id,variant_id,tss_distance,maf,ma_samples,ma_count,pval_nominal,slope,slope_se,variant_b37,variant,file,variant2,gene_id,ref,alt,gene_name
0,2323,ENSG00000237094.11,chr1_1352965_A_G_b38,851348,0.028846,15,15,0.895738,-0.036855,0.280882,1_1288345_A_G_b37,1:1288345:A:G,Stomach,1:1288345:G:A,ENSG00000237094,A,G,RP4-669L17.10
1,2323,ENSG00000225972.1,chr1_1352965_A_G_b38,723903,0.028846,15,15,0.848214,0.032457,0.169367,1_1288345_A_G_b37,1:1288345:A:G,Stomach,1:1288345:G:A,ENSG00000225972,A,G,MTND1P23
2,2323,ENSG00000225630.1,chr1_1352965_A_G_b38,723325,0.028846,15,15,0.921226,-0.021156,0.213673,1_1288345_A_G_b37,1:1288345:A:G,Stomach,1:1288345:G:A,ENSG00000225630,A,G,MTND2P28
3,2323,ENSG00000237973.1,chr1_1352965_A_G_b38,721891,0.028846,15,15,0.729151,0.060864,0.175537,1_1288345_A_G_b37,1:1288345:A:G,Stomach,1:1288345:G:A,ENSG00000237973,A,G,RP5-857K21.6
4,2323,ENSG00000229344.1,chr1_1352965_A_G_b38,720208,0.028846,15,15,0.054454,0.324285,0.167650,1_1288345_A_G_b37,1:1288345:A:G,Stomach,1:1288345:G:A,ENSG00000229344,A,G,RP5-857K21.7
